In [1]:
import pandas as pd
import sys
from bs4 import BeautifulSoup
import requests
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk.corpus import cmudict

In [4]:
data = pd.read_excel('Links.xlsx')

In [7]:
data.head()

,id,Links
0,0,https://www.politifact.com/factchecks/2023/mar...
1,1,https://www.politifact.com/factchecks/2023/mar...
2,2,https://www.politifact.com/factchecks/2023/mar...
3,3,https://www.politifact.com/factchecks/2023/mar...
4,4,https://www.politifact.com/factchecks/2023/mar...


In [6]:
data = data.drop(columns=['Unnamed: 0'])

In [10]:
#Extract URL_ID and URLs
# create an empty list to store the extracted URLs
urls = []
url_ids = []

# loop through each row in the dataframe and extract the URLs
for index, row in data.iterrows():
    url = row['Links']
    url_id = row['id']
    
    urls.append(url)
    url_ids.append(url_id)

In [14]:
#Define dictionary and stop words

#Define dictionary
positive_words = open(r'./MasterDictionary/positive-words.txt').read().lower()
negative_words = open(r'./MasterDictionary/negative-words.txt').read().lower()

#Define Stopwords
stopwords_list = open(r'./MasterDictionary/stopwords.txt').read().lower()

## Define Functions

In [32]:
#Data Scraping from URLs
def load_url_to_file(url, count):
    page = requests.get(url)
    
    print('Status code for the url: ', page.status_code)
    
    soup = BeautifulSoup(page.text, 'html.parser')
    
    heading = soup.find('title')
    #print(heading.getText())
    
    with open(f'./Scraped_data/{count}.txt', 'w+', encoding='utf-8') as file:
        file.write(str(heading.getText()))
    
    for data in soup.find_all("p"): 
        with open(f'./Scraped_data/{count}.txt', 'a+', encoding='utf-8') as file:
            file.write(f'\n{str(data.get_text())}')
            
    text = open(f'./Scraped_data/{count}.txt', 'r', encoding='utf-8').read()
    
    return text

In [16]:
#Removing punctuation
def remove_punctuation(text):
    text_punc = re.sub(r'[^\w\s]','',text)
    
    return text_punc

In [17]:
#Removing all Stopwords
def remove_stopwords(text):
    word_tok = word_tokenize(text)
    
    stoplist = stopwords_list
    
    cleanword = [word for word in word_tok if word not in stoplist]
    
    return cleanword

In [18]:
#Extracting complex words from word tokens: words with more than 2 syllables
def complex_words(word_token):
    cmu_dict = cmudict.dict()
    
    def count_syllables(word):
        """
        Count the number of syllables in a word using the CMU Pronouncing Dictionary
        """
        if word.lower() not in cmu_dict:
        # If the word is not in the CMU Pronouncing Dictionary, assume it has one syllable
            return 1
        return max([len([y for y in x if y[-1].isdigit()]) for x in cmu_dict[word.lower()]])
    
    long_words = [word for word in word_token if count_syllables(word) > 2]
    
    return long_words

### Defining metrics functions

In [19]:
positive_score = []
negative_score = []
polarity_score = []
subjectivity = []
avg_sent_length = []
percent_complex = []
fog_index = []
avg_no_word = []
complex_word_count = []
clean_word_count = []
syllable_count = []
personal_pronouns = []
avg_word_length = []

In [20]:
#Check for the formula and correct this
def positive_negative_polarity_subjectivity(pos, neg, cleanword_list):
    neg = neg*-1   
    polarity_sc = (pos - neg)/(pos + neg + 0.000001)
    
    subjectivity_sc = (pos + neg) / ((len(cleanword_list)) + 0.000001)
    
    if len(cleanword_list)==0:
        positive_score.append(0)
        negative_score.append(0)
        polarity_score.append(0)
        subjectivity.append(0)
    else:
        positive_score.append(pos)
        negative_score.append(neg)
        polarity_score.append(polarity_sc)
        subjectivity.append(subjectivity_sc)

In [21]:
def analysis_readability(cleanword_list, sent_tokens, complex_word_list):
    mean_sent_length = len(cleanword_list) / len(sent_tokens)
    percent_complex_wrd = len(complex_word_list) / len(cleanword_list)
    fog = 0.4 * (mean_sent_length + percent_complex_wrd)
    
    if len(cleanword_list)==0:
        avg_sent_length.append(0)
        percent_complex.append(0)
        fog_index.append(0)
    else:
        avg_sent_length.append(mean_sent_length)
        percent_complex.append(percent_complex_wrd)
        fog_index.append(fog)

In [22]:
def average_words_per_sentence(word_tokens, sent_tokens):
    total = len(word_tokens) / len(sent_tokens)
    
    if len(word_tokens)==0:
        avg_no_word.append(0)
    else:
        avg_no_word.append(total)

In [23]:
def complex_words_count(complex_list):   
    if not complex_list:
        complex_word_count.append(0)
    else:
        complex_word_count.append(len(complex_list))

In [24]:
def words_count(cleanword_list):
    leng = len(cleanword_list)
    
    if len(cleanword_list)==0:
        clean_word_count.append(0)
    else:
        clean_word_count.append(leng)

In [25]:
#Syllable count
def count_syllables(word):
    c = 0
    vowels = 'aeiou'
    l = re.findall(f'(?!e$)(?!es$)(?!ed$)[{vowels}]', word, re.I)
    return len(l)

In [26]:
def syllable_count_per_word(cleanword_list):
    syllable_per_word = []
    list1 = []
    for word in cleanword_list:
        list1.append(count_syllables(word))
        syllable_per_word.append(list1)
        
    syllable_sum = 0
    for i in range(len(syllable_per_word)):
        syllable_sum += syllable_per_word[i][1]
    
    syllable_count.append(int(syllable_sum/len(syllable_per_word)))

In [27]:
#Personal Pronouns
def count_personal_pronouns(text):
    pronoun_count = re.compile(r'\b(I|we|ours|my|mine|(?-i:us))\b', re.I)
    pronouns = pronoun_count.findall(text)
    
    if not(text):
        personal_pronouns.append(0)
    else:
        personal_pronouns.append(len(pronouns))

In [28]:
#Average word length
def average_word_length(word_tokens):
    c = 0
    for word in word_tokens:
        c += len(word)
        
    if len(word_tokens)==0:
        avg_word_length.append(0)
    else:
        avg_word_length.append(round(c/len(word_tokens)))

In [33]:
for ids, url in enumerate(urls, 0):
    print("Count: ", ids)
    
    load_text = load_url_to_file(f'{url}', ids)
    sans_punct = remove_punctuation(load_text)
    total_words = word_tokenize(sans_punct)
    total_sents = sent_tokenize(load_text)
    cleanword_final = remove_stopwords(sans_punct)
    complex_list = complex_words(cleanword_final)
    
    pos = neg = 0
        
    for word in cleanword_final:
        if word in positive_words:
            pos += 1
        elif word in negative_words:
            neg -= 1
            
    positive_negative_polarity_subjectivity(pos, neg, cleanword_final)
    
    analysis_readability(total_words, total_sents, complex_list)
    
    average_words_per_sentence(total_words, total_sents)
    
    complex_words_count(complex_list)
    
    words_count(cleanword_final)
    
    #syllable_count_per_word(total_words)
    syllable_count_per_word(cleanword_final)
    
    #personal pronouns
    count_personal_pronouns(sans_punct)
    
    #Average words length
    average_word_length(total_words)

Count:  0
Status code for the url:  200
Count:  1
Status code for the url:  200
Count:  2
Status code for the url:  200
Count:  3
Status code for the url:  200
Count:  4
Status code for the url:  200
Count:  5
Status code for the url:  200
Count:  6
Status code for the url:  200
Count:  7
Status code for the url:  200
Count:  8
Status code for the url:  200
Count:  9
Status code for the url:  200
Count:  10
Status code for the url:  200
Count:  11
Status code for the url:  200
Count:  12
Status code for the url:  200
Count:  13
Status code for the url:  200
Count:  14
Status code for the url:  200
Count:  15
Status code for the url:  200
Count:  16
Status code for the url:  200
Count:  17
Status code for the url:  200
Count:  18
Status code for the url:  200
Count:  19
Status code for the url:  200
Count:  20
Status code for the url:  200
Count:  21
Status code for the url:  200
Count:  22
Status code for the url:  200
Count:  23
Status code for the url:  200
Count:  24
Status code for

Count:  198
Status code for the url:  200
Count:  199
Status code for the url:  200
Count:  200
Status code for the url:  200
Count:  201
Status code for the url:  200
Count:  202
Status code for the url:  200
Count:  203
Status code for the url:  200
Count:  204
Status code for the url:  200
Count:  205
Status code for the url:  200
Count:  206
Status code for the url:  200
Count:  207
Status code for the url:  200
Count:  208
Status code for the url:  200
Count:  209
Status code for the url:  200
Count:  210
Status code for the url:  200
Count:  211
Status code for the url:  200
Count:  212
Status code for the url:  200
Count:  213
Status code for the url:  200
Count:  214
Status code for the url:  200
Count:  215
Status code for the url:  200
Count:  216
Status code for the url:  200
Count:  217
Status code for the url:  200
Count:  218
Status code for the url:  200
Count:  219
Status code for the url:  200
Count:  220
Status code for the url:  200
Count:  221
Status code for the ur

In [34]:
df_results = pd.DataFrame()

In [35]:
positive_score[-1]

70

In [36]:
polarity_score[-1]

-0.07284768163677033

In [37]:
subjectivity[-1]

0.16538882785828168

In [38]:
df_results['ids'] = url_ids
df_results['Links'] = urls
df_results['POSITIVE SCORE'] = positive_score
df_results['NEGATIVE SCORE'] = negative_score
df_results['POLARITY SCORE'] = polarity_score
df_results['SUBJECTIVITY'] = subjectivity
df_results['AVERAGE SENTENCE LENGTH'] = avg_sent_length
df_results['PERCENTAGE OF COMPLEX WORDS'] = percent_complex
df_results['FOG INDEX'] = fog_index
df_results['AVG NUMBER OF WORDS PER SENTENCE'] = avg_no_word
df_results['COMPLEX WORD COUNT'] = complex_word_count
df_results['WORD COUNT'] = clean_word_count
df_results['SYLLABLE PER WORD'] = syllable_count
df_results['PERSONAL PRONOUNS'] = personal_pronouns
df_results['AVG WORD LENGTH'] = avg_word_length

In [39]:
df_results.head()

,ids,Links,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY,AVERAGE SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,0,https://www.politifact.com/factchecks/2023/mar...,18,29,-0.234043,0.200855,24.562500,0.122137,9.873855,24.562500,48,234,1,10,5
1,1,https://www.politifact.com/factchecks/2023/mar...,40,19,0.355932,0.229572,27.882353,0.126582,11.203574,27.882353,60,257,1,10,4
2,2,https://www.politifact.com/factchecks/2023/mar...,15,10,0.200000,0.147929,24.272727,0.104869,9.751038,24.272727,28,169,2,5,5
3,3,https://www.politifact.com/factchecks/2023/mar...,18,24,-0.142857,0.146853,24.777778,0.123318,9.960438,24.777778,55,286,2,6,5
4,4,https://www.politifact.com/factchecks/2023/mar...,21,24,-0.066667,0.161290,22.944444,0.142857,9.234921,22.944444,59,279,2,8,5


In [40]:
df_results.to_excel('Output.xlsx', index=False)